# Neural Network (MLP) Algorithm for Yeast Data
## Import libraries

In [2]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from math import floor, ceil, sqrt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import time

## Preparation of the Data
### Define methods

In [3]:
def the_train_test_split(X, test_ratio = 0.2):
    if(test_ratio >= 1 or test_ratio <0):
        test_ratio = 0.2
    row, _ = X.shape
    train_count = floor(row * (1-test_ratio))
    train = X[:train_count]
    test = X[train_count:]
    return train, test

### Read Data

In [4]:
file_name = "yeast.csv"
md = pd.read_csv(file_name)

# md.dropna(inplace = True)
# md.replace('unknown', 0, inplace = True)
md.head()

,DBName,1,2,3,4,5,6,7,8,OutName
0,ADT1_YEAST,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,MIT
1,ADT2_YEAST,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,MIT
2,ADT3_YEAST,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,MIT
3,AAR2_YEAST,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,NUC
4,AATM_YEAST,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,MIT


## Prepare the data
* Shuffle the data
* Separate the input and output variables
* Seperate the data into training and test sets
* Normalize the data


In [5]:
gl = {}

In [6]:

# Shuffle the data to get more fair representative
md.reindex(np.random.permutation(md.index))

test_ratio = 0.2
X = md.values[:,1:9]
Y = md.values[:,9:]
cat = pd.unique(Y[:,0])

In [7]:
for i in range(X.shape[1]):
    X[:,i] = (X[:,i] - X[:,i].mean())/X[:,i].std()


#
y = np.zeros((len(Y), 10))
for i in range(len(Y)):
    for j in range(10):
        if cat[j] == Y[i]:
            y[i][j] = 1
        else:
            y[i][j] = 0

In [8]:
X_train, X_test = the_train_test_split(X.astype("float64"), test_ratio = test_ratio)
Y_train, Y_test = the_train_test_split(y.astype("float64"), test_ratio = test_ratio)

In [34]:
class Neural_Network():
    def __init__(self, dimension=[100,100], epochs=100, learning_rate=0.001, act_func = "sigmoid"):
        self.dim = dimension
        self.ep = epochs
        self.lr = learning_rate
        # weight 0: input to 1st hidden
        # weight 1:  1st hidden to  2nd hidden
        # weight len(dim): last hidden to output
        # in total 1+len(dim) weight matrices
        self.weights = {} # they will be defined when the input and output are given

        # actv 0: fired from input layer
        # actv 1: fired from 1st layer
        # actv 1+len(dim): fired from output layer
        self.actv = {} # activation outputs
        # z 0: input to 1st hidden
        # z 1:  1st hidden to  2nd hidden
        # z len(dim): last hidden to output
        # in total 1+len(dim) weight matrices
        self.z = {} # middle values: inputs to next layers
    def forward(self, x):
        self.actv[0] = x
        for i in range(len(self.dim)+1):
            #print("forward actv[",i,"] : ",self.actv[i])

            self.z[i] = np.dot(self.weights[i], self.actv[i])
            #print("forward z[",i,"] : ",self.z[i])

            if i == len(self.dim):
                self.actv[i+1] = self.actFunc(self.z[i], "soft" )
            else:
                self.actv[i+1] = self.actFunc(self.z[i], "sigm" )
            #print("forward actv[",i+1,"] : ",self.actv[i+1], "#")
        return self.actv[1+len(self.dim)]

    def actFunc(self, t, type="sigm"):
        if type == "sigm":
            return self.sigm(t)
        elif type=="sigm-d":
            return self.sigm(t,True)
        elif type=="soft":
            return self.softmax(t)
        else:
            return self.sigm(t)

    def sigm(self, x, derivative=False):
        if derivative:
            return (np.exp(-x))/((np.exp(-x)+1)**2)
        return 1/(1 + np.exp(-x))

    def softmax(self, x):
        exps = np.exp(x - x.max())
        return exps / np.sum(exps, axis=0)

    def back_prop(self, y_exp, y_pred):
        err = y_pred-y_exp
        w_chng = {}
        dimlen = len(self.dim)
        # calculate the update for the last weights
        w_chng[dimlen]= np.dot(err, self.actv[1+dimlen])
        # calculate changes backwardly
        for i in range(dimlen):
            # e.g. the 1st weights (i.e. w[0])
            # will be updated when i := dimlen-1
            # so that err depends on w[1], previous err and z[0]
            # and change depends on a[1]
            #print( "err: ", err.shape, " ; sw.shape: ",self.weights[dimlen-i].T.shape," ; act.shape: ",self.actFunc(self.z[dimlen-i-1], "sigm-d").shape, "; actv: ", self.actv[dimlen-i].shape )
            err = np.multiply( np.dot(self.weights[dimlen-i].T, err), self.actFunc(self.z[dimlen-i-1], "sigm-d"))
            w_chng[dimlen-i-1] = np.dot(err, self.actv[dimlen-i])
        #print("w_chng: ", type(w_chng), " : ", w_chng)
        return w_chng

    def update_weights(self, w_changes):
        # why calling items() https://stackoverflow.com/a/62173039/13555389
        global gl
        gl["w_changes"] = w_changes
        gl['wc'] = {}
        for i, chng in w_changes.items():
            for k in range( len(self.weights[i].T)):
                self.weights[i].T[k] = self.weights[i].T[k] -    self.lr*chng
                gl['wc'][k] =- self.lr*chng
                #w_mtrx = w_mtrx - self.lr*chng

    # THE FOLLOWING FUNCTION IS COPIED IN VERBATIM
    def get_accuracy(self, x_val, y_val):
        self.pred_indices = np.empty([y_val.shape[0], 2], "int" )
        print("p_i shape:",self.pred_indices.shape)
        global gl
        predictions = []
        i = 0
        for x, y in zip(x_val, y_val):
            output = self.forward(x)
            pred = np.argmax(output)
            exp = np.argmax(y)
            self.pred_indices[i][0], self.pred_indices[i][1]= pred, exp
            #print(output, " : ", pred, " :: ", y, " : ", exp)
            predictions.append(pred == exp)
            i = i+1

        #summed = sum(pred for pred in predictions) / 100.0

        gl['p'] = predictions
        gl['p_i'] = self.pred_indices
        return sum(pred for pred in gl['p'])/len(gl['p'])#np.average(summed)

    def train(self, x_train, y_train, x_test, y_test):
        # initialize weights! etc.!
        # we seperate 1st and last because they depend on the size of x_train and y_train, respectively!
        # weights from input to 1st hidden
        self.weights[0] = np.random.randn(self.dim[0], x_train.shape[1]) * np.sqrt(1. / self.dim[0])
        # set default weights to middle layer weights if there any
        dimlen = len(self.dim)
        for i in range(dimlen - 1):
            self.weights[i+1] = np.random.randn(self.dim[i+1], self.dim[i]) * np.sqrt(1. / self.dim[i+1])
            #print("train: " ,i+1, type( self.weights[i+1]))
        # last weights
        self.weights[dimlen] = np.random.randn(y_train.shape[1], self.dim[dimlen-1]) * np.sqrt(1. / y_train.shape[1])
        #print("train: " ,dimlen, type( self.weights[dimlen]))


    # train them all
        start_time = time.time()
        print("Started training!")
        for iteration in range(self.ep):
            for x,y in zip(x_train, y_train):
                output = self.forward(x)
                # print("x: ", x, "y: ", y, "o: ", np.argmax(output), " ", output)
                changes_to_w = self.back_prop(y, output)
                self.update_weights(changes_to_w)

            accuracy = self.get_accuracy(x_test, y_test)
            print('{0}th epoch, {1:.2f} seconds wasted so far, for merely {2}  accuracy'.format(
                iteration+1, time.time() - start_time, accuracy
            ))


In [33]:
xor_x= np.array([ [0, 0],
     [1, 0],
     [0, 1],
     [1, 1]
     ])
#xor_y =np.array( [ [0], [1], [1],[0]])
xor_y =np.array( [ [1, 0], [0,1], [0,1],[1,0]])

dnn = Neural_Network(dimension=[ 3 ], epochs=1000, learning_rate=1)
dnn.train(xor_x, xor_y, xor_x, xor_y)


Started training!
x:  [0 0] y:  [0] o:  0   [1.]
x:  [1 0] y:  [1] o:  0   [1.]
x:  [0 1] y:  [1] o:  0   [1.]
x:  [1 1] y:  [0] o:  0   [1.]
p_i shape: (4, 2)
1th epoch, 0.03 seconds wasted so far, for merely [0.5]  accuracy
x:  [0 0] y:  [0] o:  0   [1.]
x:  [1 0] y:  [1] o:  0   [1.]
x:  [0 1] y:  [1] o:  0   [1.]
x:  [1 1] y:  [0] o:  0   [1.]
p_i shape: (4, 2)
2th epoch, 0.05 seconds wasted so far, for merely [0.5]  accuracy
x:  [0 0] y:  [0] o:  0   [1.]
x:  [1 0] y:  [1] o:  0   [1.]
x:  [0 1] y:  [1] o:  0   [1.]
x:  [1 1] y:  [0] o:  0   [1.]
p_i shape: (4, 2)
3th epoch, 0.05 seconds wasted so far, for merely [0.5]  accuracy
x:  [0 0] y:  [0] o:  0   [1.]
x:  [1 0] y:  [1] o:  0   [1.]
x:  [0 1] y:  [1] o:  0   [1.]
x:  [1 1] y:  [0] o:  0   [1.]
p_i shape: (4, 2)
4th epoch, 0.06 seconds wasted so far, for merely [0.5]  accuracy
x:  [0 0] y:  [0] o:  0   [1.]
x:  [1 0] y:  [1] o:  0   [1.]
x:  [0 1] y:  [1] o:  0   [1.]
x:  [1 1] y:  [0] o:  0   [1.]
p_i shape: (4, 2)
5th epoc

In [11]:
gl['p']



[False, False, True, True]

In [12]:
print(sum(pred for pred in gl['p']) )
print(len(gl['p']))
print(sum(pred for pred in gl['p'])/len(gl['p']))
summed = sum(pred for pred in gl['p']) / 100.0

2
4
0.5


In [ ]:

dnn = Neural_Network(dimension=[ 15, 20 ], epochs=40, learning_rate=0.1)
dnn.train(X_train, Y_train, X_test, Y_test)

In [ ]:
#print(gl)
for i, j in gl["w_changes"].items():
    print("i: ", i, ", j: ", j)

gl['wc']

In [ ]:
# test accuracy
print(dnn.get_accuracy(X_test, Y_test))
pred_class = cat[gl['p_i'][:,0]]
exp_class = cat[gl['p_i'][:,1]]
print(exp_class)
print(pred_class)
ConfusionMatrixDisplay.from_predictions(exp_class, pred_class)


In [ ]:
# training accuracy
print(dnn.get_accuracy(X_train, Y_train))
print(Y_test.shape[0])
pred_class = cat[gl['p_i'][:,0]]
exp_class = cat[gl['p_i'][:,1]]
print(exp_class)
print(pred_class)
ConfusionMatrixDisplay.from_predictions(exp_class, pred_class)


In [ ]:
for i in range (3):
    print("type z[",i,"] : ", type(dnn.z[i]))
    print("shape z[",i,"] : ",dnn.z[i].shape)
    #print("forward z[",i,"] : ",dnn.z[i])
    print("")
    print("type actv[",i,"] : ", type(dnn.actv[i]))
    print("shape actv[",i,"] : ",dnn.actv[i].shape)
    #print("forward actv[",i,"] : ",dnn.actv[i])
    print("")

    print("type weights[",i,"] : ", type(dnn.weights[i]))
    print("shape weights[",i,"] : ",dnn.weights[i].shape)
    #print("forward weights[",i,"] : ",dnn.weights[i])
    print("-----------------\n")

i = 3
print("type actv[",i,"] : ", type(dnn.actv[i]))
print("shape actv[",i,"] : ",dnn.actv[i].shape)
print("forward actv[",i,"] : ",dnn.actv[i])
print("")

In [ ]:
print(type(dnn.weights[0]) ) #, ", shape: "dnn.weights[0].shape)
print(type(dnn.weights[1]) ) #, ", shape: "dnn.weights[1].shape)
print(type(dnn.weights[2]) ) #, ", shape: "dnn.weights[2].shape)

In [ ]:
dnn.weights[0]

In [ ]:

dnn.weights[1]

In [ ]:

dnn.weights[2]